In [1]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
  

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Configs

In [4]:
LABEL2IDX = {
    'O': 0,
    'B-DAT': 1,
    'B-PER': 2,
    'B-ORG': 3,
    'B-LOC': 4,
    'B-EVE': 5,
    'I-DAT': 6,
    'I-PER': 7,
    'I-ORG': 8,
    'I-LOC': 9,
    'I-EVE': 10
}

IDX2LABEL = {i: k for k, i in LABEL2IDX.items()}

MAX_LEN = 128
TRAIN_BATCH_SIZE = 8
EVAL_BATCH_SIZE = 8
EPOCHS = 10
NUM_CLASS = 11
LEARNING_RATE = 1e-5

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [6]:
from datasets import concatenate_datasets, load_dataset
from transformers import AutoTokenizer
import ast
model_name = 'PartAI/TookaBERT-Base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_files = "/content/drive/MyDrive/Colab Notebooks/NER-datasets/shuffled-100000-2.csv"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/463 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.59M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/145 [00:00<?, ?B/s]

In [7]:
def parse_tokens_and_labels(example):
    # Ensure 'tokens' and 'labels' are in the expected format
    if isinstance(example['tokens'], str):
        example['tokens'] = ast.literal_eval(example['tokens'])
    if isinstance(example['labels'], str):
        example['labels'] = ast.literal_eval(example['labels'])
    return example


In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"],truncation=True,max_length=512,
 is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [9]:
dataset = load_dataset('csv', data_files=data_files)
dataset = dataset['train']
# Apply the parsing function to the entire dataset
parsed_dataset = dataset.map(parse_tokens_and_labels)
# rename labels to net_tags
parsed_dataset = parsed_dataset.rename_column("labels", "ner_tags")
tokenized_dataset = parsed_dataset.map(tokenize_and_align_labels, batched=True, remove_columns=parsed_dataset.column_names,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [10]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100000
})

In [11]:
# Printing the values and lengths of each column in the first item of tokenized_dataset
print(f"Input IDs: {tokenized_dataset[0]['input_ids']}, length: {len(tokenized_dataset[0]['input_ids'])}")
print(f"Attention Mask: {tokenized_dataset[0]['attention_mask']}, length: {len(tokenized_dataset[0]['attention_mask'])}")
print(f"Labels: {tokenized_dataset[0]['labels']}, length: {len(tokenized_dataset[0]['labels'])}")

Input IDs: [2, 2941, 3249, 1598, 5725, 698, 870, 1671, 772, 708, 2938, 2497, 4497, 687, 3358, 30566, 12249, 5768, 772, 680, 687, 3358, 870, 24202, 733, 4429, 708, 1598, 5725, 5186, 698, 848, 746, 1826, 1245, 3249, 1170, 680, 968, 698, 16220, 708, 1278, 1069, 2680, 698, 848, 964, 687, 15133, 2522, 192, 691, 1775, 30566, 12249, 1024, 680, 709, 1245, 687, 1451, 1451, 11842, 772, 169, 3], length: 67
Attention Mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], length: 67
Labels: [-100, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, -100, 0, 4, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], length: 67


In [12]:
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][0]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][1]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][-2]))
 print(tokenizer.decode(tokenized_dataset[0]['input_ids'][-1]))

<s>
نحوه
.
</s>


In [13]:
print(tokenizer.convert_ids_to_tokens(tokenized_dataset[0]['input_ids']))


['<s>', '▁نحوه', '▁دعوت', '▁پیر', 'نیا', '▁از', '▁وی', '▁چنین', '▁بود', '▁که', '▁روزی', '▁نج', 'احی', '▁در', '▁منزل', '▁عبدالعلی', '▁وزیری', '▁مهمان', '▁بود', '▁و', '▁در', '▁منزل', '▁وی', '▁سنتور', '▁می\u200c', 'زده', '▁که', '▁پیر', 'نیا', '▁بلافاصله', '▁از', '▁او', '▁برای', '▁اجرای', '▁برنامه', '▁دعوت', '▁می\u200cکند', '▁و', '▁پس', '▁از', '▁امتحانی', '▁که', '▁توسط', '▁چند', '▁استاد', '▁از', '▁او', '▁می\u200cشود', '▁در', '▁گلهای', '▁شماره', '▁۲۰', '▁به', '▁اتفاق', '▁عبدالعلی', '▁وزیری', '▁شرکت', '▁و', '▁این', '▁برنامه', '▁در', '▁دستگاه', '▁دستگاه', '▁همایون', '▁بود', '▁.', '</s>']


In [14]:
# Split the combined dataset into train, validation, and test sets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_eval_split = train_test_split['test'].train_test_split(test_size=0.5)

train_dataset = train_test_split['train']
eval_dataset = train_eval_split['train']
test_dataset = train_eval_split['test']
# Print out info of each dataset
print(f"Training set: {train_dataset} samples")
print(f"Validation set: {eval_dataset} samples")
print(f"Test set: {test_dataset} samples")


Training set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 80000
}) samples
Validation set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
}) samples
Test set: Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
}) samples


In [15]:
from transformers import DataCollatorForTokenClassification
import torch
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [16]:
train_dataset.set_format("torch")
eval_dataset.set_format("torch")
test_dataset.set_format("torch")

In [17]:
train_sample = train_dataset[0]
print(type(train_sample['input_ids']))

<class 'torch.Tensor'>


In [18]:
# check the paddings with -100s
batch = data_collator([train_dataset[i] for i in range(2)])
batch["labels"]

tensor([[-100,    0,    0,    0,    0,    3,    8,    0,    3,    8,    8,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100,
         -100, -100],
        [-100,    1,    6,    6,    0, -100,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, -100]])

In [19]:
for i in range(2):
    print(train_dataset[i]["labels"])

tensor([-100,    0,    0,    0,    0,    3,    8,    0,    3,    8,    8,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100])
tensor([-100,    1,    6,    6,    0, -100,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0, -100])


In [ ]:
IDX2LABEL

{0: 'O',
 1: 'B-DAT',
 2: 'B-PER',
 3: 'B-ORG',
 4: 'B-LOC',
 5: 'B-EVE',
 6: 'I-DAT',
 7: 'I-PER',
 8: 'I-ORG',
 9: 'I-LOC',
 10: 'I-EVE'}

In [ ]:
LABEL2IDX

{'O': 0,
 'B-DAT': 1,
 'B-PER': 2,
 'B-ORG': 3,
 'B-LOC': 4,
 'B-EVE': 5,
 'I-DAT': 6,
 'I-PER': 7,
 'I-ORG': 8,
 'I-LOC': 9,
 'I-EVE': 10}

In [20]:
!pip install seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=894362ec7872964574f027a68271cacf74bbc128100c85004a7a38f7560b978a
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [21]:
label_list = [k for k,v in LABEL2IDX.items()]
label_list

['O',
 'B-DAT',
 'B-PER',
 'B-ORG',
 'B-LOC',
 'B-EVE',
 'I-DAT',
 'I-PER',
 'I-ORG',
 'I-LOC',
 'I-EVE']

In [22]:
import numpy as np
from seqeval.metrics import classification_report
import evaluate

accuracy = evaluate.load('accuracy')
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    #############
    cr = classification_report(true_labels, true_predictions,digits=4)
    fname = "report.txt"
    file = open(fname,'w')
    file.write(cr)
    file.close()
    #############
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [23]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoConfig
model_name = 'pouria82/tookaBERT-base_finetuned-2448'
config = AutoConfig.from_pretrained(model_name)
config.attention_probs_dropout_prob = 0.2
config.hidden_dropout_prob = 0.2
config.num_labels = NUM_CLASS
config.id2label = IDX2LABEL
config.label2id = LABEL2IDX
model = AutoModelForTokenClassification.from_pretrained(model_name, config=config)

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/489M [00:00<?, ?B/s]

In [24]:
samples = train_dataset[:8]
batch = data_collator([train_dataset[i] for i in range(i)])
{k: v.shape for k, v in batch.items()}


{'input_ids': torch.Size([1, 24]),
 'attention_mask': torch.Size([1, 24]),
 'labels': torch.Size([1, 24])}

In [25]:
 model.config.max_position_embeddings

512

In [26]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 80000
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})


In [27]:
LEARNING_RATE

1e-05

In [28]:
from transformers import EarlyStoppingCallback
training_args = TrainingArguments(
    "TookaBERT-base-finetuned-244880",
    learning_rate= LEARNING_RATE,
    per_device_train_batch_size= TRAIN_BATCH_SIZE,
    per_device_eval_batch_size= EVAL_BATCH_SIZE,
    num_train_epochs= 10,
    weight_decay= 0.025,#*********************
    warmup_ratio = 0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    lr_scheduler_type="linear",
    greater_is_better=True,
    push_to_hub=True,
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.157700,0.151483,0.659609,0.724209,0.690401,0.944074
2,0.145900,0.143382,0.678902,0.723384,0.700437,0.946922
3,0.124100,0.149959,0.666901,0.740715,0.701873,0.946519
4,0.115100,0.154759,0.678702,0.734342,0.705426,0.947216
5,0.100400,0.158796,0.680060,0.729482,0.703904,0.947435


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.157700,0.151483,0.659609,0.724209,0.690401,0.944074
2,0.145900,0.143382,0.678902,0.723384,0.700437,0.946922
3,0.124100,0.149959,0.666901,0.740715,0.701873,0.946519
4,0.115100,0.154759,0.678702,0.734342,0.705426,0.947216
5,0.100400,0.158796,0.680060,0.729482,0.703904,0.947435
6,0.090800,0.168266,0.681312,0.731637,0.705578,0.947165
7,0.080800,0.179990,0.676499,0.732921,0.703581,0.946484
8,0.073000,0.181676,0.688951,0.719624,0.703954,0.947616
9,0.070000,0.193088,0.680858,0.728840,0.704033,0.946648


TrainOutput(global_step=90000, training_loss=0.10777504976060655, metrics={'train_runtime': 8128.1321, 'train_samples_per_second': 98.424, 'train_steps_per_second': 12.303, 'total_flos': 2.225100874970947e+16, 'train_loss': 0.10777504976060655, 'epoch': 9.0})

In [29]:
model.config.num_labels

11

In [30]:
validation_results = trainer.evaluate()

In [31]:
validation_results

{'eval_loss': 0.16826625168323517,
 'eval_precision': 0.6813116433969515,
 'eval_recall': 0.731636863823934,
 'eval_f1': 0.7055780327651391,
 'eval_accuracy': 0.9471651402740455,
 'eval_runtime': 28.3727,
 'eval_samples_per_second': 352.451,
 'eval_steps_per_second': 44.056,
 'epoch': 9.0}

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
        )
test_results = trainer.evaluate()

In [33]:
test_results

{'eval_loss': 0.16584433615207672,
 'eval_precision': 0.6799294147304735,
 'eval_recall': 0.7357915795216877,
 'eval_f1': 0.7067583797357789,
 'eval_accuracy': 0.9476498450744493,
 'eval_runtime': 29.398,
 'eval_samples_per_second': 340.16,
 'eval_steps_per_second': 42.52}

In [34]:
with open('report.txt') as f:
    for line in f.readlines():
        print(line)

              precision    recall  f1-score   support



         DAT     0.7685    0.8090    0.7882      7596

         EVE     0.5423    0.6248    0.5806       749

         LOC     0.6318    0.6850    0.6573      6607

         ORG     0.6401    0.7073    0.6720      3017

         PER     0.6548    0.7230    0.6872      4025



   micro avg     0.6799    0.7358    0.7068     21994

   macro avg     0.6475    0.7098    0.6771     21994

weighted avg     0.6813    0.7358    0.7074     21994

